# **Simple Agent** - Testing Email Tool

In [1]:
import os
import requests
from pydantic import BaseModel, EmailStr, ValidationError, Field

# Warning control
import warnings
warnings.filterwarnings('ignore')

In [2]:
from crewai import Agent, Task, Crew

### Create single Agent:

In [3]:
single_agent = Agent(
    role="Email Agent",
    goal="You are an expert in email communication. You will send any report to our client via email in 'plain text' or 'html' format.",
    backstory=(
        "You work in a large compnay as a personal assistant to the executive team, you manage the whole teams eamil correspondance, you are a detailed oriented person. and signs all emails with: Kind Regards, Bob the email guy."
    ),
    allow_delegation=False,
    verbose=True
)

In [4]:
from crewai_tools import BaseTool

### Sentiment Analysis Tool

In [5]:
class SentimentAnalysisTool(BaseTool):
    name: str ="Sentiment Analysis Tool"
    description: str = ("Analyzes the sentiment of text "
         "to ensure positive and engaging communication.")
    
    def _run(self, text: str) -> str:
        # Your custom code tool goes here
        return "positive"

### Send Email Tool

In [6]:
import contiguity

contiguity_crewai_token = os.getenv("CONTIGUITY_CREWAI_TOKEN")
client = contiguity.login(contiguity_crewai_token, True) # Delete True to exit debug mode.

class SendEmailTool(BaseTool):
    name: str ="Send Email Tool"
    description: str = ("Use this tool whenever you need to send an email to someone. You will need to create the email_body in 'plain_text' or 'html' format as a string and pass that to teh Send Email Tool.")

    def _run(self, text: str) -> str:
        # Plain text email (To send as HTML change "text" to "html")
        email_object = {
                "to": "drjanduplessis@icloud.com",
                "from": "CrewAI Agent - Contiguity",
                "subject": "My first email!",
                "text": "I sent an email using Contiguity"
            }
        return client.send.email(email_object)
        


In [7]:
send_email_tool = SendEmailTool()

In [8]:
promo_text = """The article “Destigmatizing Borderline Personality Disorder: A Call to Action for Psychological Science” by Sara R. Masland and colleagues, published in Perspectives on Psychological Science, addresses the pervasive stigma associated with Borderline Personality Disorder (BPD) and calls for changes in how psychological science approaches this condition.

Key Points from the Article:
Historical and Gendered Stigma:
The article highlights that BPD is one of the most stigmatized psychological disorders, often linked with negative stereotypes and pejorative views. It also notes that the stigma has a gendered aspect, with BPD frequently diagnosed in women, which intersects with societal biases and gender norms.
Impact on Research and Treatment:
The authors discuss how stigma affects both individuals with BPD and the scientists studying it. This stigma can lead to biased research and editorial processes, which in turn perpetuate negative stereotypes and hinder effective treatment and understanding.

Recommendations for Change:
To combat this, the authors provide several recommendations:
For Researchers: Conduct and publish research that challenges stigmatizing views and highlights the complexity and variability of BPD.
2. For Reviewers and Editors: Ensure that review processes do not propagate stigma and that the language used in publications is respectful and non-stigmatizing.
For Educators and Clinicians: Educate about BPD in a way that emphasizes empathy and understanding, and implement treatments that are person-centered and trauma-informed.

Call to Action:
The article urges the psychological science community to take active steps to reduce stigma by integrating these recommendations into their professional practices. The goal is to foster a more inclusive and supportive environment for individuals with BPD and the professionals who support them.

Why This Matters:
Reducing stigma is crucial for improving the lives of people with BPD, ensuring they receive compassionate and effective care, and advancing scientific understanding in a way that is ethical and humane. By addressing and mitigating stigma, psychological science can better serve the public good.

For more detailed information, you can access the full article through academic databases or platforms like Perspectives on Psychological Science and researcher-specific portals ￼ ￼.
"""

In [9]:
# Define a task for sending the email
email_task = Task(
    description='Send an report on AI Today by email to {email_address}',
    expected_output='Email sent confirmation',
    agent=single_agent,
    tools=['send_email_tool']

)

## Set-up Crew

In [10]:
crew = Crew(
    agents=[single_agent],
    tasks=[email_task],
    verbose=2,
	memory=True
)

In [11]:
# Define your inputs
inputs = {
    "email_address": "jan.duplessis@nhs.net",
    "email_subject": "CrewAI Email Agent",
    "email_body": "from inputs"
}

result = crew.kickoff(inputs=inputs)
print(result)

 [DEBUG]: == Working Agent: Email Agent
 [INFO]: == Starting Task: Send an report on AI Today by email to jan.duplessis@nhs.net


KeyError: 'tools'

In [12]:
from IPython.display import Markdown
Markdown(result)

NameError: name 'result' is not defined